In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
CD = pd.read_csv('Company_Data.csv')
CD

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


In [3]:
CD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [4]:
CD['Sales'].min()
CD['Sales'].max()

16.27

In [5]:
np.median(CD["Sales"])

7.49

In [6]:
CD["sales"]="<=7.49"
CD.loc[CD["Sales"]>=7.49,"sales"]=">=7.49"

In [7]:
CD["sales"].value_counts()


>=7.49    201
<=7.49    199
Name: sales, dtype: int64

In [8]:
##Dropping Sales column from the data 
#CD.drop(["Sales"],axis=1,inplace = True)
CD.replace({'sales':{'>=7.49':1,'<=7.49':0}}, inplace=True)
CD

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes,1
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes,1
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes,1
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes,0
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes,1
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes,0
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes,0
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes,0


In [9]:
CD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
 11  sales        400 non-null    int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 37.6+ KB


In [10]:
#scaling
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
CD["Urban"]= le.fit_transform(CD["Urban"])
CD["US"]= le.fit_transform(CD["US"])
CD["ShelveLoc"]= le.fit_transform(CD["ShelveLoc"])


In [11]:
CD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    int32  
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    int32  
 10  US           400 non-null    int32  
 11  sales        400 non-null    int64  
dtypes: float64(1), int32(3), int64(8)
memory usage: 32.9 KB


In [12]:
x= CD.iloc[:,1:11]
y = CD.iloc[:,11]

In [13]:
colnames = list(CD.columns)
colnames

['Sales',
 'CompPrice',
 'Income',
 'Advertising',
 'Population',
 'Price',
 'ShelveLoc',
 'Age',
 'Education',
 'Urban',
 'US',
 'sales']

In [14]:
#splitting data into training and test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3, random_state= 40)

## building random forest classifier

In [15]:
num_trees = 100
max_features = 3

In [16]:
kfold = KFold(n_splits=10,shuffle=True, random_state=7)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
model.fit(x_train,y_train)
results = cross_val_score(model, x, y, cv=kfold)
print(results.mean())

0.78


In [17]:
pred_train = model.predict(x_train)


In [18]:
from sklearn.metrics import accuracy_score
accuracy_train = accuracy_score(y_train,pred_train)

In [19]:
accuracy_train #100%

1.0

In [20]:
##Confusion matrix
from sklearn.metrics import confusion_matrix
con_train = confusion_matrix(y_train,pred_train)

In [21]:
con_train

array([[132,   0],
       [  0, 148]], dtype=int64)

In [22]:
pred_test = model.predict(x_test)

In [23]:
accuracy_test = accuracy_score(y_test,pred_test)

In [24]:
accuracy_test

0.75

In [25]:
np.mean(y_test==pred_test)

0.75

In [26]:
con_test = confusion_matrix(y_test,pred_test)

In [27]:
con_test

array([[50, 17],
       [13, 40]], dtype=int64)

In [28]:
#visualizing
from sklearn.tree import export_graphviz 
from six import StringIO
import pydotplus
colnames = list(CD.columns)
predictors = colnames[0:10]
target = colnames[10]
tree1 = model.estimators_[20]
dot_data = StringIO()
export_graphviz(tree1,out_file = dot_data, feature_names =predictors, class_names = target, filled =True,
                rounded=True,impurity =False,proportion=False,precision =2)

In [29]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())


In [30]:

##Creating pdf file
graph.write_pdf('companyrf.pdf')


True

In [32]:
##Creating png file
graph.write_png('companyrf.png')

True